In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt

In [2]:
communications = pd.read_csv('communications.csv', sep=';')
course_employee_sms = pd.read_csv('course_employee_sms.csv', sep=';')
employees = pd.read_csv('employees.csv', sep=';')
courses_passing = pd.read_csv('courses_passing.csv', sep=';')
courses_info = pd.read_csv('courses_info.csv', sep=';')

In [3]:
communications['communication_dt'] = pd.to_datetime(communications['communication_dt'])
courses_passing['start_dt'] = pd.to_datetime(courses_passing['start_dt'])
courses_passing['last_activity_dt'] = pd.to_datetime(courses_passing['last_activity_dt'])
courses_passing['end_dt'] = pd.to_datetime(courses_passing['end_dt'])

In [4]:
communications_prep = communications.groupby(['employee_id','communication_dt'])['communication_score'].mean().reset_index()

In [5]:
res = communications_prep.merge(courses_passing[courses_passing.pass_frac==1], how='left', on='employee_id')
res = res[res['course_id'].notnull()]
res['before_course'] = res[['end_dt', 'communication_dt']].apply(lambda x: 1 if x.end_dt < x.communication_dt else 0, axis=1)

In [6]:
test5 = res.groupby(['course_id','before_course','employee_id'])['communication_score'].mean().reset_index()

In [7]:
test6 = test5[test5['before_course']==1].merge(test5[test5['before_course']==0], how='left', on=['employee_id','course_id'])

In [8]:
test6['diff']=test6['communication_score_x']-test6['communication_score_y']

In [9]:
test6['target'] = test6['diff'] #.apply(lambda x: 1 if x>2 else 0)

In [10]:
new_df = test6.merge(res[['course_id','employee_id','end_dt']].drop_duplicates(), how='left', on=['employee_id','course_id'])[['course_id','employee_id','target','end_dt']] 

In [11]:
new_df = new_df.merge(courses_info, how='left', on='course_id')

In [12]:
new_df = new_df.rename(columns={'employee_id':'user_id', 'course_nm':'title', 'target':'rating','end_dt':'unix_timestamp'})

In [13]:
new_df.head(3)

,course_id,user_id,rating,unix_timestamp,title
0,0.0,0c048b3a-434e-49e6-55c1-247efb389cec,6.228863,2023-03-24,Основы клиентского сервиса: Практический подход
1,0.0,1543843a-4723-ed2a-b08e-18053ae6dc5b,2.075101,2023-11-05,Основы клиентского сервиса: Практический подход
2,0.0,1c383cd3-0b7c-298a-b502-93adfecb7b18,5.857167,2023-04-06,Основы клиентского сервиса: Практический подход


In [14]:
new_df.to_csv('new_df.csv')